# Predicting Energy Behavior of Prosumers in Estonia - A Project on Minimizing Imbalance Costs by Enefit

_Objective : Develop a prediction model for prosumer energy patterns in Estonia to minimize future imbalance costs._

## Data Wrangling

<blockquote> Compiling a dataset with relevant features : county name, consumption pattern, production pattern, installed and capacity of solar panel</blockquote>

### 1. Load Data and Initial Analysis

In [4]:
#Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Train Data : Consumption and Production Pattern year 2021 - 2023**

In [6]:
df1 = pd.read_csv('C:/Users/Michelle Natali/Downloads/Group 4 - Data/predict-energy-behavior-of-prosumers/train.csv')
df1

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60


In [7]:
#Getting Information from the Data
df1.info()
df1.dtypes
df1.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   datetime            object 
 6   data_block_id       int64  
 7   row_id              int64  
 8   prediction_unit_id  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB


(2018352, 9)

In [8]:
df1.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id'],
      dtype='object')

**Client Data : Installed Capacity of Solar Panel**

In [9]:
df2 = pd.read_csv('C:/Users/Michelle Natali/Downloads/Group 4 - Data/predict-energy-behavior-of-prosumers/client.csv')
df2

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2
...,...,...,...,...,...,...,...
41914,1,15,51,415.60,0,2023-05-29,637
41915,3,15,161,2035.75,0,2023-05-29,637
41916,0,15,15,620.00,1,2023-05-29,637
41917,1,15,20,624.50,1,2023-05-29,637


In [10]:
#Getting Information from the Data
df2.info()
df2.dtypes
df2.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41919 entries, 0 to 41918
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_type        41919 non-null  int64  
 1   county              41919 non-null  int64  
 2   eic_count           41919 non-null  int64  
 3   installed_capacity  41919 non-null  float64
 4   is_business         41919 non-null  int64  
 5   date                41919 non-null  object 
 6   data_block_id       41919 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 2.2+ MB


(41919, 7)

**County Names**

In [11]:
CountyName = pd.read_json('C:/Users/Michelle Natali/Downloads/Group 4 - Data/predict-energy-behavior-of-prosumers/county_id_to_name_map.json',orient='index')
CountyName

CountyName['county'] = CountyName.index
CountyName['county_name'] = CountyName[0]
CountyName=CountyName[['county','county_name']]
CountyName

,county,county_name
0,0,HARJUMAA
1,1,HIIUMAA
2,2,IDA-VIRUMAA
3,3,JÄRVAMAA
4,4,JÕGEVAMAA
5,5,LÄÄNE-VIRUMAA
6,6,LÄÄNEMAA
7,7,PÄRNUMAA
8,8,PÕLVAMAA
9,9,RAPLAMAA


### 2. Data Merging

In [14]:
# Splitting the Date and Time to Merge Train and Client Table
df1[['date', 'time']] = df1['datetime'].astype(str).str.split(' ',n=1, expand=True)
df1.drop(columns='time', inplace=True)
df1

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01
...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023-05-31


In [15]:
# Merging Train and Client Table
merged_df = pd.merge(df1, df2, on=['county','is_business','product_type','date'], how='left')
merged_df

,county,is_business,product_type,target,is_consumption,datetime,data_block_id_x,row_id,prediction_unit_id,date,eic_count,installed_capacity,data_block_id_y
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01,108.0,952.89,2.0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01,108.0,952.89,2.0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01,17.0,166.40,2.0
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01,17.0,166.40,2.0
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01,688.0,7207.88,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31,NaN,NaN,NaN
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31,NaN,NaN,NaN
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31,NaN,NaN,NaN
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023-05-31,NaN,NaN,NaN


In [18]:
# Merging Train, Client and County Names
merged_df3 = pd.merge(merged_df,CountyName, on=['county'], how = 'left')
merged_df3

,county,is_business,product_type,target,is_consumption,datetime,data_block_id_x,row_id,prediction_unit_id,date,eic_count,installed_capacity,data_block_id_y,county_name
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01,108.0,952.89,2.0,HARJUMAA
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01,108.0,952.89,2.0,HARJUMAA
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01,17.0,166.40,2.0,HARJUMAA
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01,17.0,166.40,2.0,HARJUMAA
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01,688.0,7207.88,2.0,HARJUMAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31,NaN,NaN,NaN,VÕRUMAA
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31,NaN,NaN,NaN,VÕRUMAA
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31,NaN,NaN,NaN,VÕRUMAA
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023-05-31,NaN,NaN,NaN,VÕRUMAA


In [19]:
#Re arrange the Data
merged_df3.rename(columns={'data_block_id_x': 'data_block_id'}, inplace=True)
desired_columns = ['county', 'county_name', 'is_business', 'datetime', 'is_consumption', 'target', 'installed_capacity', 'product_type','row_id','data_block_id']
reordered_df = merged_df3[desired_columns]
reordered_df

,county,county_name,is_business,datetime,is_consumption,target,installed_capacity,product_type,row_id,data_block_id
0,0,HARJUMAA,0,2021-09-01 00:00:00,0,0.713,952.89,1,0,0
1,0,HARJUMAA,0,2021-09-01 00:00:00,1,96.590,952.89,1,1,0
2,0,HARJUMAA,0,2021-09-01 00:00:00,0,0.000,166.40,2,2,0
3,0,HARJUMAA,0,2021-09-01 00:00:00,1,17.314,166.40,2,3,0
4,0,HARJUMAA,0,2021-09-01 00:00:00,0,2.904,7207.88,3,4,0
...,...,...,...,...,...,...,...,...,...,...
2018347,15,VÕRUMAA,1,2023-05-31 23:00:00,1,197.233,NaN,0,2018347,637
2018348,15,VÕRUMAA,1,2023-05-31 23:00:00,0,0.000,NaN,1,2018348,637
2018349,15,VÕRUMAA,1,2023-05-31 23:00:00,1,28.404,NaN,1,2018349,637
2018350,15,VÕRUMAA,1,2023-05-31 23:00:00,0,0.000,NaN,3,2018350,637


### 3. Data Cleaning

In [20]:
# Counting the missing data in each column of Dataframe
reordered_df.isnull().sum()

county                   0
county_name              0
is_business              0
datetime                 0
is_consumption           0
target                 528
installed_capacity    6240
product_type             0
row_id                   0
data_block_id            0
dtype: int64

In [52]:
#Drop rows containing NaN values
cleaned_df = reordered_df.dropna()
df4 = cleaned_df

#Saving the file into Excel files
df4.to_csv('C:/Users/Michelle Natali/Downloads/Group 4 - Data/predict-energy-behavior-of-prosumers/merged_production_consumption_installed_capacity_target_and_datetime.csv')

df4

,county,county_name,is_business,datetime,is_consumption,target,installed_capacity,product_type,row_id,data_block_id
0,0,HARJUMAA,0,2021-09-01 00:00:00,0,0.713,952.89,1,0,0
1,0,HARJUMAA,0,2021-09-01 00:00:00,1,96.590,952.89,1,1,0
2,0,HARJUMAA,0,2021-09-01 00:00:00,0,0.000,166.40,2,2,0
3,0,HARJUMAA,0,2021-09-01 00:00:00,1,17.314,166.40,2,3,0
4,0,HARJUMAA,0,2021-09-01 00:00:00,0,2.904,7207.88,3,4,0
...,...,...,...,...,...,...,...,...,...,...
2012107,15,VÕRUMAA,1,2023-05-29 23:00:00,1,188.167,620.00,0,2012107,635
2012108,15,VÕRUMAA,1,2023-05-29 23:00:00,0,0.000,624.50,1,2012108,635
2012109,15,VÕRUMAA,1,2023-05-29 23:00:00,1,31.484,624.50,1,2012109,635
2012110,15,VÕRUMAA,1,2023-05-29 23:00:00,0,0.000,2188.20,3,2012110,635


In [29]:
# Counting the missing data in each column of Dataframe
df4.isnull().sum()

county                0
county_name           0
is_business           0
datetime              0
is_consumption        0
target                0
installed_capacity    0
product_type          0
row_id                0
data_block_id         0
dtype: int64

In [30]:
# Understanding the distribution of prosumer types
unique_prosumer_types = df4.groupby(['county','county_name', 'is_business', 'product_type']).size().reset_index(name='count')
unique_prosumer_types

,county,county_name,is_business,product_type,count
0,0,HARJUMAA,0,1,30520
1,0,HARJUMAA,0,2,30520
2,0,HARJUMAA,0,3,30520
3,0,HARJUMAA,1,0,30520
4,0,HARJUMAA,1,1,30520
...,...,...,...,...,...
64,15,VÕRUMAA,0,1,30520
65,15,VÕRUMAA,0,3,30520
66,15,VÕRUMAA,1,0,26154
67,15,VÕRUMAA,1,1,30520


### 4.Processing Data (Statistics, Separation)

In [33]:
print(len(cleaned_df.query("is_consumption==0")['target'].values))
print(len(cleaned_df.query("is_consumption==1")['target'].values))

print(cleaned_df['is_consumption'].value_counts())

1005792
1005792
is_consumption
0    1005792
1    1005792
Name: count, dtype: int64


In [55]:
#Separating Production and Consumption
temp_table_0 = cleaned_df.drop(columns=['target','is_consumption','row_id']).drop_duplicates()
temp_table_0['production'] = cleaned_df.loc[cleaned_df['is_consumption'] == 0]['target'].values
temp_table_0['production'] = cleaned_df.loc[cleaned_df['is_consumption'] == 1]['target'].values
target_data_0 = temp_table_0

#Convert Datetime
target_data_0['datetime'] = target_data_0['datetime'].values.astype('datetime64[h]')
target_data_0

,county,county_name,is_business,datetime,installed_capacity,product_type,data_block_id,production
0,0,HARJUMAA,0,2021-09-01 00:00:00,952.89,1,0,96.590
2,0,HARJUMAA,0,2021-09-01 00:00:00,166.40,2,0,17.314
4,0,HARJUMAA,0,2021-09-01 00:00:00,7207.88,3,0,656.859
6,0,HARJUMAA,1,2021-09-01 00:00:00,400.00,0,0,59.000
8,0,HARJUMAA,1,2021-09-01 00:00:00,1411.00,1,0,501.760
...,...,...,...,...,...,...,...,...
2012102,15,VÕRUMAA,0,2023-05-29 23:00:00,415.60,1,635,37.058
2012104,15,VÕRUMAA,0,2023-05-29 23:00:00,2035.75,3,635,110.621
2012106,15,VÕRUMAA,1,2023-05-29 23:00:00,620.00,0,635,188.167
2012108,15,VÕRUMAA,1,2023-05-29 23:00:00,624.50,1,635,31.484


In [71]:
#Saving the file into Excel files
target_data_0.to_csv('C:/Users/Michelle Natali/Downloads/Group 4 - Data/predict-energy-behavior-of-prosumers/merged_production_consumption_installed_capacity_and_datetime.csv')

In [72]:
#Using Mean Data
mean_production_0 = target_data_0.groupby('county_name')['production'].mean().reset_index()
mean_production_0
mean_consumption_0 = target_data_0.groupby('county_name')['consumption'].mean().reset_index()
mean_consumption_0
mean_installed_0 = target_data_0.groupby('county_name')['installed_capacity'].mean().reset_index()
mean_installed_0

mean_per_county = target_data_0.groupby('county_name')[['production','consumption','installed_capacity']].mean().reset_index()
mean_per_county

,county_name,production,consumption,installed_capacity
0,HARJUMAA,253.743401,1627.268521,4508.874273
1,HIIUMAA,28.588876,47.549969,336.702519
2,IDA-VIRUMAA,30.422034,229.487286,440.726938
3,JÄRVAMAA,60.769803,264.056113,1019.876673
4,JÕGEVAMAA,44.699181,250.374396,822.486111
5,LÄÄNE-VIRUMAA,81.474715,380.391008,1327.616536
6,LÄÄNEMAA,9.848641,174.683576,465.133047
7,PÄRNUMAA,88.763351,320.123717,1491.972078
8,PÕLVAMAA,54.725353,121.150277,817.148413
9,RAPLAMAA,56.084307,123.719961,742.388425


In [73]:
mean_per_county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   county_name         16 non-null     object 
 1   production          16 non-null     float64
 2   consumption         16 non-null     float64
 3   installed_capacity  16 non-null     float64
dtypes: float64(3), object(1)
memory usage: 644.0+ bytes


In [74]:
#Saving the file into Excel files
mean_per_county.to_csv('C:/Users/Michelle Natali/Downloads/Group 4 - Data/predict-energy-behavior-of-prosumers/merged_production_consumption_and_installed_capacity.csv')